В этом блокноте приведены решения задач к третьему уроку по курсу ВШЭ "Эконометрика" на [coursera](https://www.coursera.org/learn/ekonometrika/home/week/3). Темы урока: вычисление вариаций для прогнозов регрессии, дамми-переменные, сравнение ограниченных и неограниченных моделей, проверка на лишние и недостающие регрессоры.

In [1]:
import numpy as np
import pandas as pd

## Задача на нахождение вариации для среднего и для прогноза

**Задача**

По 2040 наблюдениям оценена модель зависимости стоимости цены квартиры $price_i$ (в 1000) от метража жилой площади $livesp_i$: $price_i=−90+1.8livesp_i$. При построении 95% доверительного интервала для $E\left(price_f\mid livesp=70\right)$, чему равна $\hat{Var}\left(\hat{price_f}\mid X\right)$, eсли $\sigma^2=1259.265$, а ковариационная матрица имеет следующий вид: $\hat{Var}\left(\hat{\beta}\mid X\right)=\begin{pmatrix}21.9 & -0.46 \\ -0.46 & 0.01 \end{pmatrix}$.

Чему равна $\hat(Var(price_f - \hat{price_f}\mid X)?

**Решение**

Для значения регрессора $livesp = 70$ получаем такую модель регрессии: $price=\beta_1 + \beta_2 \cdot 70$. По формулам для вариации суммы величин получаем: $\hat{Var}(\hat{price}) = Var(\beta_1) + 70^2 \cdot Var(\beta_2) + 2\cdot 70 \cdot cov(\beta_1 \beta_2) = 6.5$

$\hat{Var}(price_f - \hat{price_f}\mid X)$ - это вариация для цены случайной квартиры, она на $\sigma^2$ больше, чем вариация для среднего значения цены: $\hat{Var}(price_f - \hat{price_f}\mid X) = 6.5 + 1259.265 = 1265.675$.

In [101]:
sigma = 1259.265
var1 = 21.9
var2 = 0.01
cov12 = -0.46
l = 70
var_y_mean = var1 + l**2 * var2 + 2*l*cov12
var_y_pred = sigma + var1 + 70**2 * var2 + 2*70*cov12
round(var_y_mean, 2), round(var_y_pred, 3)

(6.5, 1265.765)

## Бриллианты

Загружаем данные по бриллиантам (датасет `diamonds` из R).

In [128]:
df = pd.read_csv('diamonds.csv', index_col=0)
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58,335,4.34,4.35,2.75


In [129]:
len(df.columns), len(df.index)

(10, 53940)

### Простая оценка модели

**Задача**

Оцените модель $price_i=\beta_1 + \beta_2 \log(carat_i)+\epsilon_i$. С ростом количества карат на 1% на сколько долларов растёт цена?

**Решение**

Для составления моделей используем пакет `statsmodels`, необходимые функции см. ниже. По умолчанию в регрессию не входит константа, поэтому в функцию `OLS` надо передавать не просто столбцы с признаками, а результат функции `statsmodels.api.add_constant`, которая добавляет к матрице признаков столбец из единиц.

Поскольку в качестве регрессора используется логарифм числа карат, то для ответа на вопрос задачи надо узнать коэффициент при этом регрессоре и разделить его на 100.

In [130]:
import statsmodels.api as sm
model = sm.OLS(df.price, sm.add_constant(df.carat.apply(np.log)))
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.732
Model:                            OLS   Adj. R-squared:                  0.732
Method:                 Least Squares   F-statistic:                 1.473e+05
Date:                Sat, 13 Aug 2016   Prob (F-statistic):               0.00
Time:                        21:43:10   Log-Likelihood:            -4.8827e+05
No. Observations:               53940   AIC:                         9.765e+05
Df Residuals:                   53938   BIC:                         9.766e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       6237.8366     10.732    581.228      0.0

### Проверка гипотезы о значимости регрессии

**Задача**

Оцените модель $price_i=\beta_1+\beta_2 carat_i+\beta_3 x_i+\epsilon_i$. Проверьте гипотезу о значимости регрессии в целом на уровне значимости 1%.

**Решение**

Гипотеза о значимости регрессии проверяется F-тестом. Если значение F-статистики большое, то нулевая гипотеза (о том, что все коэффициенты равны 0) отвергается. Также можно посмотреть p-значения при отдельных коэффициентах: если их значения меньше, чем уровень значимости (0.01), то они являются значимыми.

In [131]:
  results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                 1.570e+05
Date:                Sat, 13 Aug 2016   Prob (F-statistic):               0.00
Time:                        21:43:13   Log-Likelihood:            -4.7199e+05
No. Observations:               53940   AIC:                         9.440e+05
Df Residuals:                   53937   BIC:                         9.440e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1737.9497    103.623     16.772      0.0

### Нахождение других параметров модели

**Задача**

Чему равен скорректированный $R^2_adj$ в модели $price_i=\beta_1+\beta_2 carat_i+\epsilon_i$?

Оцените следующую модель: $price_i=\beta_1+\beta_2 carat_i+\epsilon_i$. Чему равен AIC

**Решение**

Все нужные параметры можно посмотреть в `summary` результатов оценки. Также эти значения лежат в `model.fit()`, их можно посмотреть просто через точку.

In [132]:
model = sm.OLS(df.price, sm.add_constant(df.carat))
results = model.fit()
print('R2 adj = %.2f, AIC = %.2f' % (results.rsquared_adj, results.aic))
print(results.summary())

R2 adj = 0.85, AIC = 945464.53
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                 3.041e+05
Date:                Sat, 13 Aug 2016   Prob (F-statistic):               0.00
Time:                        21:43:16   Log-Likelihood:            -4.7273e+05
No. Observations:               53940   AIC:                         9.455e+05
Df Residuals:                   53938   BIC:                         9.455e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const      -2256.3606

### Сравнение нескольких моделей

**Задача**

Оцените следующие три модели на данных по бриллиантам:
$$price_i=\beta_1+\beta_2 carat_i+\epsilon_i$$,
$$price_i=\beta_1+\beta_2 carat_i+\beta_3 depth_i + \epsilon_i$$,
$$price_i=\beta_1+\beta_2 carat_i+\beta_3 depth_i + \beta_4 cut_i + \epsilon_i$$

Какая модель лучше по критерию AIC?

**Решение**

Главная проблема в том, что признак `cut` (огранка бриллианта) - категориальный, его надо факторизовать. Для этого надо ввести дамми-переменные. Это можно сделать средствами `pandas` функцией `pandas.get_dummies`. Она вернет 5 столбцов (по количеству разных значений в факторизуемом столбце) с нулями и единицами. Чтобы избежать мультиколлинеарности (линейной зависимости признаков), на вход модели подаем не 5 столбцов, а только 4 (в примере ниже убран столбец `Fair`).

Второй вариант. Можно использовать модуль `statsmodels.formula.api`, в котором можно задавать регрессию формулой (синтаксис похож на R). Тогда можно не делать факторизацию вручную, а просто указать в формуле, что данная переменная - категориальная. При оценке одна из дамми-переменных будет исключена из регрессии автоматически. Другие примеры использования этого модуля можно найти [здесь](http://statsmodels.sourceforge.net/devel/example_formulas.html).

In [134]:
df = df.join(pd.get_dummies(df.cut))

In [147]:
cols = list(df.cut.unique())
model1 = sm.OLS(df.price, sm.add_constant(df[['carat']]))
model2 = sm.OLS(df.price, sm.add_constant(df[['carat', 'depth']]))
model3 = sm.OLS(df.price, sm.add_constant(df[['carat', 'depth'] + cols[:-1]]))
res1 = model1.fit()
res2 = model2.fit()
res3 = model3.fit()
print('AIC1 = %d, AIC2 = %d, AIC3 = %d' % (res1.aic, res2.aic, res3.aic))

AIC1 = 945464, AIC2 = 944982, AIC3 = 942746


In [148]:
import statsmodels.formula.api as sfm
model3_f = sfm.ols('price ~ carat + depth + C(cut)', data=df)
res3_f = model3_f.fit()
print('AIC = ', res3_f.aic)
print(res3_f.summary())

AIC =  942746.145422
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.857
Method:                 Least Squares   F-statistic:                 5.377e+04
Date:                Sat, 13 Aug 2016   Prob (F-statistic):               0.00
Time:                        21:55:46   Log-Likelihood:            -4.7137e+05
No. Observations:               53940   AIC:                         9.427e+05
Df Residuals:                   53933   BIC:                         9.428e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
Intercept    

### Проверка гипотезы о незначимости регрессоров (об ограничениях)

**Задача**

Оцените следующие две модели:
$$price_i=\beta_1+\beta_2 carat_i+\epsilon_i$$,
$$price_i=\beta_1+\beta_2 carat_i+\beta_3 depth_i+\beta_4 cut_i+\epsilon_i$$.

Проведите тест на два линейных ограничения в R. Чему равно F-значение?

**Решение**

Надо оценить, имеет ли смысл вводить дополнительные переменные в регрессию или же можно обойтись и меньшим числом переменных без особой потери в качестве. Для этого проводится F-тест: считается значение F-статистики по формуле
$$F = \frac{\left(R^2_{UR}-R^2_R\right)/r}{\left(1 - R^2_{UR}\right) / (n-k)}$$, где $r$ - количество переменных, незначимость которых мы проверяем (количество ограничений), $n$ - количество наблюдений, $k$ - количество коэффициентов в неограниченной модели. Индексы UR относятся к неограниченной модели (с большим числом регрессоров), индексы $R$ - к ограниченной модели. Если значение этой статистики большое (больше F-критического при нужном уровне значимости), то нулевая гипотеза о том, что коэффициенты при этих регрессорах равны 0 отвергается.

Значение F можно посчитать вручную, а можно воспользоваться функциями `wald_test` или `f_test` от результатов оценки. В качестве параметра туда надо передать матрицу: число строк равно числу ограничений, число столбцов равно числу коэффициентов в неограниченной модели, в самих строках на месте проверяемого коэффициента стоит 1, на остальных местах - 0. Вид матрицы приведен отдельно ниже. 

При решении надо учесть, что для регрессора `cut` создано 4 дамми-переменных.

In [149]:
r_ur = res3.rsquared
r_r = res1.rsquared
r = 5
n = len(df.index)
k = 7
print(((r_ur - r_r) / r) / ((1 - r_ur) / (n - k)))

559.641097328


In [162]:
print(res3.params)
print(np.eye(len(res3.params))[2:])
print(res3.wald_test(np.eye(len(res3.params))[2:]))
print(res3.f_test(np.eye(len(res3.params))[2:]))

const        -648.907301
carat        7873.248664
depth         -50.417619
Ideal        1684.079947
Premium      1299.401230
Good         1036.268784
Very Good    1398.556215
dtype: float64
[[ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]]
<F test: F=array([[ 559.64109733]]), p=0.0, df_denom=53933, df_num=5>
<F test: F=array([[ 559.64109733]]), p=0.0, df_denom=53933, df_num=5>


### Проверка пропуска переменнных (тест Рамсея)

**Задача**

Если провести тест Рамсея для модели $price_i=\beta_1+\beta_2 carat_i+\beta_3 depth_i+\beta_4 cut_i+\epsilon_i$ на уровне значимости 1%, что он покажет?

**Решение**

Тест Рамсея проверяет, не пропущены ли какие-то переменные при построении регрессии (иными словами, достаточно ли имеющихся переменных для качественного описания или чего-то не хватает). Идея этого теста в следующем. Если для описания искомой переменной нам не хватает регрессоров, то часть необъясненной информации осталась в искомой переменной $y$. Значит, если мы добавим в регрессию новые слагаемые, которые являются степенями $y$, то качество регрессии должно увеличиться. Значит, можно сделать новую модель, в которую добавить, например, регрессоры $y^2$ и $y^3$, а затем проверить гипотезу о незначимости этих регрессоров. Если эта гипотеза отвергнута, значит, в исходной регрессии переменных недостаточно.

**Я не нашел готовую функцию в `statsmodels`, которая осуществляет тест Рамсея (другое название - RESET-тест)**. Поэтому вручную добавим в модель регрессоры $y^2$ и $y^3$, а затем проведем `wald_test`. Если в результате теста получится довольно большое значение F-статистики (критическое значение на 95% уровне значимости - примерно 3), то нулевая гипотеза отвергается и регрессоров недостаточно.

In [164]:
df['price2'] = df.price ** 2
df['price3'] = df.price ** 3

In [165]:
model4 = sm.OLS(df.price, sm.add_constant(df[['carat', 'depth'] + cols[:-1] + ['price2', 'price3']]))
res4 = model4.fit()

In [166]:
print(res4.wald_test(np.eye(len(res4.params))[-2:]))

<F test: F=array([[ 318541.30937788]]), p=0.0, df_denom=53931, df_num=2>
